# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: **

The goal of this project is to identify studendts who need to be called to interventio or not. Technically, we have two possible outcomes, Yes: Call the student & No: Don't need to call the student. The ultmate goal is going to be finding a deision boundary which will seperate the data as possible fail students versus possible not fail students. Thus, this is a supervised CLASSIFICATION problem due to the Boolean outputs (Yes or No). 

A supervised regression problem, on the other hand, contains continuous data and the ultimate goal is trying to find a best fit line which is irrelevant with the goal of this project. 

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [1]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("student-data.csv")
print ("Student data read successfully!")

Student data read successfully!


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [3]:
# TODO: Calculate number of students
n_students = len(student_data.index)

# TODO: Calculate number of features
n_features = len(student_data.columns)-1

# TODO: Calculate passing students
n_passed = len(student_data[(student_data.passed == 'yes')])

# TODO: Calculate failing students
n_failed = len(student_data[(student_data.passed == 'no')])

# TODO: Calculate graduation rate
grad_rate = float(n_passed)/float(n_students)*100.0

# Print the results
print ("Total number of students: {}".format(n_students))
print ("Number of features: {}".format(n_features))
print ("Number of students who passed: {}".format(n_passed))
print ("Number of students who failed: {}".format(n_failed))
print ("Graduation rate of the class: {:.2f}%".format(grad_rate))

Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [4]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print ("Feature columns:\n{}".format(feature_cols))
print ("\nTarget column: {}".format(target_col))

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print ("\nFeature values:")
print (X_all.head())

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [5]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print ("Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns)))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [6]:
# TODO: Import any additional functionality you may need here
from sklearn.cross_validation import train_test_split

# TODO: Set the number of training points
num_train = 300

# Set the number of testing points
num_test = X_all.shape[0] - num_train

# TODO: Shuffle and split the dataset into the number of training and testing points above
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=num_test, train_size=num_train, random_state=0)

# Show the results of the split
print ("Training set has {} samples.".format(X_train.shape[0]))
print ("Testing set has {} samples.".format(X_test.shape[0]))

Training set has 300 samples.
Testing set has 95 samples.


C:\Users\AHMETF\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

**The following supervised learning models are currently available in** [`scikit-learn`](http://scikit-learn.org/stable/supervised_learning.html) **that you may choose from:**
- Gaussian Naive Bayes (GaussianNB)
- Decision Trees
- Ensemble Methods (Bagging, AdaBoost, Random Forest, Gradient Boosting)
- K-Nearest Neighbors (KNeighbors)
- Stochastic Gradient Descent (SGDC)
- Support Vector Machines (SVM)
- Logistic Regression

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. For each model chosen*
- Describe one real-world application in industry where the model can be applied. *(You may need to do a small bit of research for this — give references!)* 
- What are the strengths of the model; when does it perform well? 
- What are the weaknesses of the model; when does it perform poorly?
- What makes this model a good candidate for the problem, given what you know about the data?

# **Answer: **



In [87]:
Decision Trees Classifier:

Decision tree classifiers are appealing models if we are willing to commentate. As the name decision 
tree offers, we can think of this model as splitting our data by making decisions based on asking a 
bunch of questions. 

Using the decision tree algorithm, we start at the tree root and seperate the data on the feature that
results in the largest information gain (IG). In an iterative process, we can then repeat this 
seperation process at each child node until the leaves are branchless. That is the samples at each 
node all belong to the same class. In practice, this can result in a very deep tree with many nodes,
which can easily lead to overfitting. Thus, we typically want to prune the tree by setting a limit for 
the maximal depth of the tree.

Advantages:

Easy to understand.
Able to handle descrete inputs.
Able to handle continuous inputs using decision boundaries.
Using Information Gain, decides how to split the decision boundaries for the dataset.

Disadvantages:

Exponentially growing nodes.
When maximum depth is too high, the model is prone to overfit. 
Bad performance when ID3 (Iterative Dichotomiser 3) algorithm could not split the data near by the top.

Refs & Apps:
    
1- Kamiński, B.; Jakubczyk, M.; Szufel, P. (2017). "A framework for sensitivity analysis of decision 
trees". Central European Journal of Operations Research. doi:10.1007/s10100-017-0479-6
2- Quinlan, J. R. (1987). "Simplifying decision trees". International Journal of Man-Machine Studies.
27 (3): 221. doi:10.1016/S0020-7373(87)80053-6
3- Utgoff, P. E. (1989). Incremental induction of decision trees. Machine learning, 4(2), 161-186. 
doi:10.1023/A:1022699900025
4- A. ERCIL. Classification trees prove useful in nondestructive testing of spotweld quality.Welding 
Journal, 72(9):59, September 1993. Issue Title: Special emphasis: Rebuilding America's roads, railways
and bridges.
5- Nicholas Weir, S. Djorgovski, & Usama M. Fayyad. Initial galaxy counts from digitized POSS-II. The
Astronomical Journal, 110(1):1, 1995.
6- Nicholas Weir, Usama M. Fayyad, & S. Djorgovski. Automated star/galaxy classification for digitized
POSS-II. The Astronomical Journal, 109(6):2401, 1995.
7- Sebastian Raschka, Python Machine Learning, September 23, 2015


SyntaxError: invalid syntax (<ipython-input-87-42f245afd2b6>, line 1)

In [70]:
Support Vector Machine Classifier:

Support vector machine (SVM), which can be esteemed as a generaliation of the perceptron. Using the perceptron
algorithm, we minimized misclassification errors. However, in SVMs, our optimization target is to maximize 
the margin. The margin is defined as the distance between the separating hyperplane (decision boundary) 
and the training samples that are closest to this hyperplane, which are the so-called support
vectors.

Advantages:

They are powerful kernels. 
SVMs avoid overfitting by choosing a specific hyperplane among the many that can separate the data in
the feature space. 
SVMs find the maximum margin hyperplane, the hyperplane that maximixes the minimum distance from the 
hyperplane to the closest training point.

Disadvantage:

Overfitting is a problem when we have too many features. 
The biggest limitation of the support vector approach lies in choice of the kernel.

Application: Classification of Images

Refs & Apps: 
[1] Antonio Carlos Gay Thome,  SVM Classifiers – Concepts and Applications to Character Recognition.
Advances in Character Recognition, Edited by Xiaoqing Ding, ISBN 978-953-51-0823-8, 240 pages, Publisher:
InTech, Chapters published November 07, 2012 under CC BY 3.0 license DOI: 10.5772/2575.
[2] Burges, "A Tutorial on Support Vector Machines for Pattern Recognition", Data Mining and Knowledge 
Discovery, vol. 2, pp. 121-167, 2016.
[3]Priyanka Patil1, S. I. Nipanikar, Text Recognition By Using Character Descriptor And SVM Classifier.
International Journal of Advanced Research in  Electrical, Electronics and Instrumentation Engineering. 
[4] Sadhana Tiwari, Character Recognition using Ensemble classifier. International Journal of Computer
Science & Engineering Technology (IJCSET)
[5] Sebastian Raschka, Python Machine Learning, September 23, 2015

SyntaxError: invalid syntax (<ipython-input-70-f9562c058367>, line 1)

In [69]:
Random Forest Classifier (as an Ensemble Method):
    
The random forest algorithm can be summarized in four simple steps:
1. Draw a random bootstrap sample of size n (randomly choose n samples from the training set with replacement).
2. Grow a decision tree from the bootstrap sample. At each node:
   a) Randomly select d features without replacement.
   b) Split the node using the feature that provides the best split according to the objective function,
    for instance, by maximizing the information gain.
3. Repeat the steps 1 to 2 k times.
4. Aggregate the prediction by each tree to assign the class label by majority vote.

Although random forests don't offer the same level of interpretability as decision trees, a big advantage
of random forests is that we don't have to worry so much about choosing good hyperparameter values. We
typically don't need to prune the random forest since the ensemble model is quite robust to noise from 
the individual decision trees.

Application:
    
Internet traffic interception
Video/image/voice classification


Advantages:
    
It scales quickly. 
They have ability to deal with unbalanced and missing data.

Disadvantages:
    
Overfits if there is noise .

Ref:
[1] L. Breiman, “Random Forests,” Machine Learning, Vol. 45, No. 1, 2001, pp. 5-32. doi:10.1023/A:1010933404324.
[2] Xiao Di Tian HanBo-Suk YangSoo-Jong Lee, Application of Random Forest Algorithm in Machine Fault Diagnosis. https://doi.org/10.1007/978-1-84628-814-2_82
[3] Sebastian Raschka, Python Machine Learning, September 23, 2015






SyntaxError: invalid syntax (<ipython-input-69-6b2e18a3169a>, line 1)

### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [11]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print ("Trained model in {:.4f} seconds".format(end - start))

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print ("Made predictions in {:.4f} seconds.".format(end - start))
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print ("Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train)))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print ("F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train)))
    print ("F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test)))

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [16]:
# TODO: Import the three supervised learning models from sklearn
# from sklearn import model_A
# from sklearn import model_B
# from sklearn import model_C
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

# TODO: Initialize the three models
clf_A = DecisionTreeClassifier(random_state=0)
clf_B = SVC(random_state=1)
clf_C = RandomForestClassifier(random_state=2)

# TODO: Set up the training set sizes
X_train_100 = X_train[:100]
y_train_100 = y_train[:100]

X_train_200 = X_train[:200]
y_train_200 = y_train[:200]

X_train_300 = X_train
y_train_300 = y_train

# TODO: Execute the 'train_predict' function for each classifier and each training set size
# train_predict(clf, X_train, y_train, X_test, y_test)

for clf in [clf_A, clf_B, clf_C]:
    for size in [100, 200, 300]:
        train_predict(clf, X_train[:size], y_train[:size], X_test, y_test)
        print ('\n')

Training a DecisionTreeClassifier using a training set size of 100. . .
Trained model in 0.0030 seconds
Made predictions in 0.0000 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0010 seconds.
F1 score for test set: 0.6942.


Training a DecisionTreeClassifier using a training set size of 200. . .
Trained model in 0.0020 seconds
Made predictions in 0.0010 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0010 seconds.
F1 score for test set: 0.7132.


Training a DecisionTreeClassifier using a training set size of 300. . .
Trained model in 0.0040 seconds
Made predictions in 0.0000 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0010 seconds.
F1 score for test set: 0.7167.


Training a SVC using a training set size of 100. . .
Trained model in 0.0020 seconds
Made predictions in 0.0020 seconds.
F1 score for training set: 0.8591.
Made predictions in 0.0010 seconds.
F1 score for test set: 0.7838.


Training a SVC using a training set size of

### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

** Classifer 1 - Decision Tree Classifier**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |  0.0020 seconds         |  0.0010 seconds        |  1.0000          |  0.6942         |
| 200               |  0.0020 seconds         |  0.0005 seconds        |  1.0000          |  0.7132         |
| 300               |  0.0025 seconds         |  0.0005 seconds        |  1.0000          |  0.7167
      |

** Classifer 2 - Support Vector Machine Classifier**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |  0.0025                 |  0.0010                |  0.8591          |  0.7838         |
| 200               |  0.0055                 |  0.0015                |  0.8693          |  0.7755         |
| 300               |  0.0095                 |  0.0020                |  0.8692          |  0.7586         |

** Classifer 3 - Random Forest Classifier**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |  0.0140                 |  0.0025                |  0.9764          |  0.7031         |
| 200               |  0.0195                 |  0.0025                |  0.9925          |  0.6829         |
| 300               |  0.0255                 |  0.0020                |  0.9903          |  0.7852         |

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Choosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: **

I want to say Random Forest Classifier is the best model due to the F1-score = 0.7852 for the test set, eventhough it is depending on a faster model of the Decision Trees Classifier. Considering the F1-score for the test set of Decision Trees is lower than RM but the F1-score for the training set tells us the model is likely overfitting to the training set. Thus, when we optimize the model to decide the decision boundary this overfitting will mislead us.  

### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. Be sure that you are describing the major qualities of the model, such as how the model is trained and how the model makes a prediction. Avoid using advanced mathematical or technical jargon, such as describing equations or discussing the algorithm implementation.*

**Answer: **

I am going to make a similiar explanation as I did before at the Charity ML project.

Random Forest technique manages the overfitting problem when we run into decision trees. Using the training samples, it grows multiple classification trees. At the time of prediction, each tree is used to address a prediction and every result is counted as a vote. The forest chooses the classification having the most votes (over all the trees in the forest).
In particular, trees that are grown very deep tend to learn highly irregular patterns: they overfit their training sets, i.e. have low bias, but very high variance. Random forests are a way of averaging multiple deep decision trees, trained on different parts of the same training set, with the goal of reducing the variance.

The forest error rate depends on two things: 
1-The correlation between any two trees in the forest. An increase on the correlation increases the forest error rate. 
2-The strength of each individual tree in the forest. A tree with a low error rate is a strong classifier. Increasing the strength of the individual trees decreases the forest error rate.

Assume that we have a student at the school named Arthur and we want to know whether Arthur is going to be summoned or not to an early intervention. Now let's say Nancy is one of the officials who make decision about the interventions. When we ask to Nancy about Arthur's situation she will first look at the data which includes features that a given person having some features will be and another person passing over such features will not. She will then endeavour to generalize a pattern from the previous data that she has already learned and will start to ask questions about Arthur such as "What is his GPA?", "How many days he was absent?" and so on, like game of 20 questions. At the end of the mini survey, Nancy could simply tell Arthur's situation, Yes (CALL him) or No (NOT CALL him). The decsion making process of Nancy is look like a decision tree where each question that she asked about him is a node of the tree and answers to each question is a path in the tree that will end in either YES or NO at the corresponding leaf.


Let's we have an employed person in CA named Arthur and we want to know wheter Arthur will donate or not? Now assume that we have Charity ML employee, named Nancy. When we ask to Nancy about Arthur's decision she will first look at the data which includes features that a given person having some features donates and another person passing over such features does not. She will then endeavour to generalize a pattern from the previous data that she has already learned and will start to ask questions to Arthur such as "What's your highest degree?", "Are you married?" and so on, like game of 20 questions. At the end of the mini survey, Nancy could simply tell Arthur's situation, Yes or No. The decsion making process of Nancy is look like a decision tree where each question that she asked to him is a node of the tree and answers to each question is a path in the tree that will end in either YES or NO at the corresponding leaf. Now we have a decision tree but it is pron to errors since it is only depending on Nancy's survey. To correct Nancy, let's say officials X, Y and Z also asked the same question as "Will Artur be summoned or not?" In order to make more accurate results, let's also give them a slightly different trainig data set instead of the same data set that Nancy has already used before. Then they will ask different questions about Arthur to get a conclusion. This process will reduce the error rate as they all asked different sets of questions about him. If the majority says Arthur is positive as a possible to fail his graduation then we can assure that he is from the set of the individuals who will likely to be summoned and we may send them a notice of call. After the voting, we decide the final conclusion. All of the previous ensemble of trees create a forest and since the data that all been used is random, the algorithm is called Random Forrest Algorithm.


Ref: http://blog.echen.me/2011/03/14/laymans-introduction-to-random-forests/

### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.GridSearchCV`](http://scikit-learn.org/0.17/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [84]:
# TODO: Import 'GridSearchCV' and 'make_scorer'

from sklearn.grid_search import GridSearchCV
from sklearn.metrics import f1_score, make_scorer
from sklearn.ensemble import RandomForestClassifier

# Create the parameters list you wish to tune

# Create the parameters list you wish to tune
parameters = {'random_state':[0,1,3,5,7,11,13,15],
              'max_depth':[1,2,3,4],
              'criterion':['gini','entropy']}

# Initialize the classifier
clf = RandomForestClassifier(random_state=42)

# Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(f1_score, pos_label='yes')

# Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = GridSearchCV(clf, param_grid=parameters, scoring=f1_scorer)

# Fit the grid search object to the training data and find the optimal parameters
grid_obj = grid_obj.fit(X_train, y_train)

# Get the estimator
clf = grid_obj.best_estimator_

# Review best parameters
print (grid_obj.best_params_)
print ("---")




# Report the final F1 score for training and testing after parameter tuning
print ("Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train)))
print ("Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test)))

{'criterion': 'gini', 'max_depth': 4, 'random_state': 5}
---
Made predictions in 0.0020 seconds.
Tuned model has a training F1 score of 0.8470.
Made predictions in 0.0020 seconds.
Tuned model has a testing F1 score of 0.7639.


### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

**Answer: **

UNTUNED MODEL

F1 score for training set: 0.9903.

Made predictions in 0.0030 seconds.

F1 score for test set: 0.7852.

Made predictions in 0.0020 seconds.


TUNED MODEL

Tuned model has a training F1 score of 0.8470.

Made predictions in 0.0020 seconds.

Tuned model has a testing F1 score of 0.7639.

Made predictions in 0.0020 seconds.


The difference on training F1-scores between tuned and untuned models is 0.1433. This progress from tuning the max_depth parameter reduced the overfitting to the training data. Moreover, we also reduced the testing F1-score 0.0213 more after tunning. The prediction times for training is 0.0010 faster after tuned while for testing are the same in both case. For Random Forest Clsifiers we have  limited number of parameters can be tuned. Overall, I can see selecting Random Forest Classifiers as an objective
is a wise decision.


> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.